In [1]:
import socket
import sys
import os
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
from pyspark.sql import Row, SparkSession
import time 

In [5]:
# 전역변수 설정 
SPARK_APP_NAME = "DATA-Preparation"
SPARK_MASTER = "local[3]"
PORT = 9999
HOST_NAME = socket.gethostname()
os.environ['HADOOP_HOME'] = '/hadoop-3.2.2'
os.environ['SPARK_HOME'] = '/spark-home'
os.environ['hadoop.home.dir'] = '/hadoop-3.2.2/bin'
os.environ['CLASSPATH'] = '$CLASSPATH:/hadoop-3.2.2/spark-3.2.0-bin-hadoop3.2.tar'
print(os.getenv('HADOOP_HOME'))

# Define path
DATA_ROOT = '/tf/notebooks/data'
DATA_PATH = f'/{DATA_ROOT}'

/hadoop-3.2.2


In [6]:
# Spark session 생성 메서드 
def init_session():
    spark = SparkSession.builder.master(SPARK_MASTER).appName(SPARK_APP_NAME).config('spark.driver.host', HOST_NAME).getOrCreate()
    default_conf = spark.sparkContext._conf.getAll()
    conf = spark.sparkContext._conf.setAll([
        ('spark.executor.instances', 1)
        , ('spark.driver.memory', '12g'), ('spark.executor.memory', '8g'), ('spark.driver.maxResultSize', '8g')
        , ('spark.driver.allowMultipleContexts', 'true'), ('spark.sql.shuffle.partitions', 8)
        #,('spark.memory.offHeap.enabled', True), ('spark.memory.offHeap.size', '8g')
    ])
    spark.sparkContext.stop()
    
    spark = SparkSession.builder.master(SPARK_MASTER).appName(SPARK_APP_NAME).config(conf=default_conf).getOrCreate()
    new_conf = spark.SparkContext._conf.getAll()
    print(f'Updated Conf : {new_conf}')
    return spark
    

In [7]:
# 스파크 생성 
!echo $SPARK_HOME
spark = init_session()
sc = spark.sparkContext
print(sc._conf.getAll())
sc

/spark-home


FileNotFoundError: [Errno 2] No such file or directory: '/spark-home/./bin/spark-submit': '/spark-home/./bin/spark-submit'

In [ ]:
# channel, day, seg, setop, 1000

# setop data n 개 만들기
def create_setops():
    setop_count = 10000
    setop_name = ['ST_A', 'ST_B', 'ST_C', 'ST_D', 'ST_E']
    setops = []
    for s in setop_name:
        for i in range(0, int(setop_count/len(setop_name))):
            setops.append(f'{s}_{i:03d}')
            
    print(setops[-10:])
    return setops

# channel, day, seg data 생성 
def create_others():
    # 20
    channels = ['KBS', 'MBC', 'SBS', 'JTBC', 'CBS' ,  'OCN', 'TVN', 'TVCH', 'BTN', 'EBS',  'Arirang', 'JTV', 'GAME-TV', 'HBC', 'BBC',  'CNN', 'CNBC', 'CCN', 'NHK', 'ABC'] 
    days = ['mon', 'tue', 'wed', 'thu', 'fri', 'sat', 'sun']
    hour_bands = ['00', '01', '10', '11', '23']
    segs = ['Agriculture', 'Game']
    rows = []
    row = []
    for c in channels:
        for d in days:
            for s in segs:
                row = [c, d, s]
                rows.append(row)
                
    print(rows[:5])
    return rows

def merge_to_inventory(setops, rows):
    inven_time = 10000 # 하드코딩 시간
    invens = []
    for r in rows:
        for s in setops:
            invens.append(r + [s, inven_time])
    print(f'Inventory Length : {len(invens):,}')
    return invens

def define_schema():
    